In [20]:
import pandas as pd
import numpy as np
import pickle
import warnings
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from xgboost import XGBRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error


trail = pd.read_csv(r'C:\Users\Karthick Palanivel\Documents\GitHub\Price_prediction\data\final.csv')

# removing outliers in price and cleaning postal code
trail = trail[trail['Price'] < 7.1e5]
trail = trail[trail['postal code'].str.len() <= 4] 

trail = trail.drop(trail[trail['Number of frontages']==14].index) # after romving,min = 6
trail = trail.drop(trail[trail['Toilets'] > 9].index)

# since bathrooms and shower rooms are same adding it and removing >10 and (-1) values considering it as outliers 
trail['Bathrooms']=trail['Bathrooms']+trail['Shower rooms'] 
trail = trail.drop(trail[trail['Bathrooms'] > 10].index)
trail = trail.drop(trail[trail['Bathrooms'] == -1].index)

# replacing catagorical names in 'type of property'
trail['Type of property'] = trail['Type of property'].replace('new-real-estate-project-apartments', 'apartment')
trail['Type of property'] = trail['Type of property'].replace('new-real-estate-project-houses', 'house')
trail['Type of property'] = trail['Type of property'].replace('apartment-block', 'apartment')

trail = trail.drop(columns=['Address', 'Location', 'Furnished', 'immo code','Construction year','Terrace', 'Office' ,'Primary energy consumption','Terrace surface','Outdoor parking space','Shower rooms'], axis=1)


column=[ 'Type of property', 'Building condition', 'Kitchen type',  'province','Energy class', 'Heating type',] # catagorical
column1 = ['Surface of the plot', 'Living room surface', ] # scaling to  be 
column3= ['Number of frontages','Bedrooms','Bathrooms', 'Toilets', 'Price', 'postal code', 'Furnished'] # as it is


In [21]:
ohe = OneHotEncoder(handle_unknown='ignore', sparse=True)
one_hot_encoded_array = ohe.fit_transform(trail[column]).toarray()
print(one_hot_encoded_array)

[[0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [22]:
ohe.categories_

[array(['apartment', 'bungalow', 'castle', 'chalet', 'country-cottage',
        'duplex', 'exceptional-property', 'farmhouse', 'flat-studio',
        'ground-floor', 'house', 'kot', 'loft', 'manor-house', 'mansion',
        'mixed-use-building', 'other-property', 'penthouse',
        'service-flat', 'town-house', 'triplex', 'villa'], dtype=object),
 array(['0', 'As new', 'Good', 'Just renovated', 'To be done up',
        'To renovate', 'To restore'], dtype=object),
 array(['0', 'Hyper equipped', 'Installed', 'Not installed',
        'Semi equipped', 'USA hyper equipped', 'USA installed',
        'USA semi equipped', 'USA uninstalled'], dtype=object),
 array(['Antwerp', 'Brussels Capital Region', 'East Flanders',
        'Flemish Brabant', 'Hainaut (West)', 'Limburg', 'Liège',
        'Luxembourg (shared with Eastern Hainaut)', 'Namur',
        'Walloon Brabant', 'West Flanders', nan], dtype=object),
 array(['A', 'A+', 'A++', 'B', 'C', 'C_B', 'D', 'E', 'F', 'G', 'G_F', 'NS'],
       dty

In [23]:
categories = np.concatenate(ohe.categories_)
print(categories)

['apartment' 'bungalow' 'castle' 'chalet' 'country-cottage' 'duplex'
 'exceptional-property' 'farmhouse' 'flat-studio' 'ground-floor' 'house'
 'kot' 'loft' 'manor-house' 'mansion' 'mixed-use-building'
 'other-property' 'penthouse' 'service-flat' 'town-house' 'triplex'
 'villa' '0' 'As new' 'Good' 'Just renovated' 'To be done up'
 'To renovate' 'To restore' '0' 'Hyper equipped' 'Installed'
 'Not installed' 'Semi equipped' 'USA hyper equipped' 'USA installed'
 'USA semi equipped' 'USA uninstalled' 'Antwerp' 'Brussels Capital Region'
 'East Flanders' 'Flemish Brabant' 'Hainaut (West)' 'Limburg' 'Liège'
 'Luxembourg (shared with Eastern Hainaut)' 'Namur' 'Walloon Brabant'
 'West Flanders' nan 'A' 'A+' 'A++' 'B' 'C' 'C_B' 'D' 'E' 'F' 'G' 'G_F'
 'NS' '0' 'Carbon' 'Electric' 'Fuel oil' 'Gas' 'Pellet' 'Solar' 'Wood']


In [24]:
#creating dataframe with enoded data and removing unwanted columns
encoded_dataframe = pd.DataFrame(one_hot_encoded_array, columns=categories)
encoded_dataframe = encoded_dataframe.drop(columns=['0', 'NS', 'C_B', 'G_F'])
encoded_dataframe

,apartment,bungalow,castle,chalet,country-cottage,duplex,exceptional-property,farmhouse,flat-studio,ground-floor,...,E,F,G,Carbon,Electric,Fuel oil,Gas,Pellet,Solar,Wood
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7562,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
7563,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7564,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
7565,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
#removing encoded columns from trail
trail = trail.drop(columns=[ 'Type of property', 'Building condition', 'Kitchen type',  'province','Energy class', 'Heating type',]).reset_index(drop=True)
trail['postal code']= trail['postal code'].astype('int64')

In [26]:
# Models

def LinearReg(train_x,train_y, test_x,test_y):

    print('working in Linear Regression')
    # initialising 'linear regression' and creating a model
    linear = LinearRegression()
    linear.fit(train_x,train_y)

    # score of train
    train_score = linear.score(train_x,train_y)
    print("Train Score:", train_score)

    # score of test
    test_score = linear.score(test_x,test_y)
    print("Test Score:", test_score)

    print('\n')
    print('-----------------------------')
    print('\n')

    # saving the model
    filename = 'linear_pickle.pickle'
    pickle.dump(linear, open(filename, "wb"))


def DecisionTreeReg(train_x,train_y, test_x,test_y):

    print('working in Decision Tree Regression')

    #max_depth=7, min_samples_leaf=4, min_samples_split=10
    DT = DecisionTreeRegressor(max_depth=10,  min_samples_leaf=10, min_samples_split=10)
    DT.fit(train_x,train_y)

    # score of train
    train_score = DT.score(train_x,train_y)
    print("Train Score:", train_score)

    # score of test
    test_score = DT.score(test_x,test_y)
    print("Test Score:", test_score)

    print('\n')
    print('-----------------------------')
    print('\n')

    # saving the model
    filename = 'DT_pickle.pickle'
    pickle.dump(DT, open(filename, "wb"))

def NeighborsReg(train_x,train_y, test_x,test_y):

    print('working in Neighbors Regression')
    knn = KNeighborsRegressor(n_neighbors=3)
    knn.fit(train_x,train_y)

    # score of train
    train_score = knn.score(train_x,train_y)
    print("Train Score:", train_score)

    # score of test
    test_score = knn.score(test_x,test_y)
    print("Test Score:", test_score)

    print('\n')
    print('-----------------------------')
    print('\n')

    # saving the model
    filename = 'knn_pickle.pickle'
    pickle.dump(knn, open(filename, "wb"))

def RandomForestReg(train_x,train_y, test_x,test_y):

    print('working in Random Forest Regressor')
   
    y_train = np.ravel(train_y)
    y_test = np.ravel(test_y)
    forest = RandomForestRegressor()
    forest.fit(train_x,train_y)

    # score of train
    train_score = forest.score(train_x,train_y)
    print("Train Score:", train_score)

    # score of test
    test_score = forest.score(test_x,test_y)
    print("Test Score:", test_score)

    print('\n')
    print('-----------------------------')
    print('\n')

    # saving the model
    filename = 'forest_pickle.pickle'
    pickle.dump(forest, open(filename, "wb"))

"""
# load model
loaded_model = pickle.load(open(filename, "rb"))

# you can use loaded model to compute predictions
y_predicted = loaded_model.predict(X)
"""

def xgboostReg(train_x,train_y, test_x,test_y):

    print('working in xgboost')
    xgb_regressor = XGBRegressor()
    xgb_regressor.fit(train_x,train_y)

    Y_pred_train = xgb_regressor.predict(train_x)
    Y_pred_test = xgb_regressor.predict(test_x)

    train_mse = mean_squared_error(train_y, Y_pred_train)
    test_mse = mean_squared_error(test_y, Y_pred_test)

    # score of train
    train_score = xgb_regressor.score(train_x,train_y)
    print("Train Score:", train_score)

    # score of test
    test_score = xgb_regressor.score(test_x,test_y)
    print("Test Score:", test_score)

    print("Train MSE:", train_mse)
    print("Test MSE:", test_mse)
    
    print('\n')
    print('-----------------------------')
    print('\n')

    # saving the model
    filename = 'xgb_reg_pickle.pickle'
    pickle.dump(xgb_regressor, open(filename, "wb"))


In [27]:
trail

,postal code,Price,Bedrooms,Surface of the plot,Living room surface,Number of frontages,Bathrooms,Toilets
0,2910,469000.0,3,760,34,4,1,2
1,6700,285000.0,2,0,0,2,2,2
2,6700,285000.0,2,0,0,2,2,2
3,2000,425000.0,2,0,40,0,1,0
4,4460,189000.0,2,360,32,2,1,0
...,...,...,...,...,...,...,...,...
7562,2460,465000.0,2,1221,45,4,1,2
7563,3120,499000.0,3,17656,0,4,1,2
7564,3360,450000.0,3,0,0,2,2,0
7565,1420,699000.0,3,0,0,3,2,0


In [28]:
# merging trail(original) and encoded dataframe
new = pd.concat([trail,encoded_dataframe], axis=1)
new

,postal code,Price,Bedrooms,Surface of the plot,Living room surface,Number of frontages,Bathrooms,Toilets,apartment,bungalow,...,E,F,G,Carbon,Electric,Fuel oil,Gas,Pellet,Solar,Wood
0,2910,469000.0,3,760,34,4,1,2,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,6700,285000.0,2,0,0,2,2,2,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,6700,285000.0,2,0,0,2,2,2,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,2000,425000.0,2,0,40,0,1,0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4460,189000.0,2,360,32,2,1,0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7562,2460,465000.0,2,1221,45,4,1,2,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
7563,3120,499000.0,3,17656,0,4,1,2,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7564,3360,450000.0,3,0,0,2,2,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
7565,1420,699000.0,3,0,0,3,2,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [29]:
#initialising 
x = new.drop(['Price'], axis=1)
y = new['Price']
x_train,x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)


# normalisation or scaling - train and test
minmax_scaler = MinMaxScaler()
x_train[['Surface of the plot', 'Living room surface']] = minmax_scaler.fit_transform(x_train[[ 'Surface of the plot', 'Living room surface']])
x_test[[ 'Surface of the plot', 'Living room surface']]= minmax_scaler.transform(x_test[[ 'Surface of the plot', 'Living room surface']])

display(x_train.head())

,postal code,Bedrooms,Surface of the plot,Living room surface,Number of frontages,Bathrooms,Toilets,apartment,bungalow,castle,...,E,F,G,Carbon,Electric,Fuel oil,Gas,Pellet,Solar,Wood
4553,8370,1,0.000000,0.000000,2,1,1,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1905,3800,3,0.004588,0.125000,0,1,1,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3333,1130,3,0.006210,0.053571,2,2,2,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6452,1190,2,0.000000,0.110714,2,1,1,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
6620,9690,2,0.004283,0.000000,3,1,1,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [30]:
warnings.filterwarnings("ignore")

LinearReg(x_train,y_train,x_test,y_test)
DecisionTreeReg(x_train,y_train,x_test,y_test)
NeighborsReg(x_train,y_train,x_test,y_test)
xgboostReg(x_train,y_train,x_test,y_test)
RandomForestReg(x_train,y_train,x_test,y_test)

working in Linear Regression
Train Score: 0.40340673522226844
Test Score: 0.38203904429757496


-----------------------------


working in Decision Tree Regression
Train Score: 0.575222783506262
Test Score: 0.36873645735607274


-----------------------------


working in Neighbors Regression
Train Score: 0.6381334428027151
Test Score: 0.21354393994929388


-----------------------------


working in xgboost
Train Score: 0.8680146801484602
Test Score: 0.5463227051480206
Train MSE: 2870693802.6044407
Test MSE: 9961680562.313295


-----------------------------


working in Random Forest Regressor
Train Score: 0.9341812770812973
Test Score: 0.5181705460201542


-----------------------------




In [31]:
'''app = FastAPI()
pickle_in = open ('forest_pickle', 'rb')
s = pickle.load(pickle_in)'''

"app = FastAPI()\npickle_in = open ('forest_pickle', 'rb')\ns = pickle.load(pickle_in)"